先运行第一部分，保存下cookie

In [ ]:
import json
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.twitter.com/login")

# 等待手动登录
input("请在浏览器中手动登录，然后在这里按回车继续...")

# 保存Cookie到文件
with open("twitter_cookies.json", "w") as file:
    json.dump(driver.get_cookies(), file)

driver.quit()


从下面的部分开始，只要报错，就重启Jupyter内核，然后从下面的代码开始运行。注意，df.to_excel('Part_2.xlsx')中的名字需要切换

In [1]:
import time
from selenium import webdriver
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 设置WebDriver
driver = webdriver.Chrome()

driver.get("https://www.twitter.com")

# 从文件加载Cookie
with open("twitter_cookies.json", "r") as file:
    cookies = json.load(file)
    for cookie in cookies:
        driver.add_cookie(cookie)

# 初始化DataFrame
df = pd.DataFrame(columns=['Tweet Date', 'Tweet Text'])

# remaining_dates = missing_dates.copy()
remaining_dates_df = pd.read_csv('remaining_dates.csv')
remaining_dates = pd.DatetimeIndex(remaining_dates_df.iloc[:,0])

# 遍历日期
for current_date in remaining_dates:
    until = current_date.strftime('%Y-%m-%d')
    since = (current_date - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    print("Start " + str(until))
    print("Remain:" + str(len(remaining_dates)))
    # 构建并访问URL
    url = f"https://twitter.com/search?f=top&q=(%23Bitcoin)%20lang%3Aen%20until%3A{until}%20since%3A{since}&src=typed_query"
    driver.get(url)

    # 初始化WebDriverWait对象
    wait = WebDriverWait(driver, 100)
    wait.until(EC.presence_of_element_located((By.XPATH, '//article[@data-testid="tweet"]')))

    max_scrolls = 2  # 设置一个最大滚动次数
    scroll_count = 0  # 初始化滚动计数器

    # 模拟滚动以加载更多推文
    while True:
        # 当前推文数量
        start_count = len(driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]'))
        print("start_count:", start_count)

        # 滚动到页面底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 短暂等待页面加载更多推文
        time.sleep(10)  # 等待3秒

        # 再次检查推文数量
        end_count = len(driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]'))
        print("end_count:", end_count)

        if end_count == start_count:
            scroll_count += 1

        # 如果已加载的推文数量达到或超过10条，或滚动次数达到最大值，则停止滚动
        if end_count >= 10 or scroll_count >= max_scrolls:
            break
    
    if start_count <3:
        break
    
    if end_count < 10:
        continue
        
    # 提取推文和链接
    tweet_elements = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    for tweet_element in tweet_elements[:10]:  # 获取前10条推文
        try:
            tweet_text = tweet_element.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
            # 创建一个新的DataFrame来存储这一行数据
            new_row = pd.DataFrame({'Tweet Date': [current_date.strftime('%Y-%m-%d')], 'Tweet Text': [tweet_text]})
            # 使用concat合并新的DataFrame
            df = pd.concat([df, new_row], ignore_index=True)
        except Exception as e:
            print("Error extracting tweet:", e)

    print("Successfully for" + str(current_date))
    remaining_dates = remaining_dates.drop(current_date)
    time.sleep(20)

# 关闭WebDriver
driver.quit()

Start 2022-09-23
Remain:9
start_count: 3
end_count: 13
Successfully for2022-09-23 00:00:00
Start 2022-08-09
Remain:8
start_count: 5
end_count: 11
Successfully for2022-08-09 00:00:00
Start 2022-04-27
Remain:7
start_count: 7
end_count: 12
Successfully for2022-04-27 00:00:00
Start 2022-11-07
Remain:6
start_count: 6
end_count: 15
Successfully for2022-11-07 00:00:00
Start 2022-09-20
Remain:5
start_count: 5
end_count: 11
Successfully for2022-09-20 00:00:00
Start 2022-09-19
Remain:4
start_count: 5
end_count: 14
Successfully for2022-09-19 00:00:00
Start 2021-10-26
Remain:3
start_count: 4
end_count: 11
Successfully for2021-10-26 00:00:00
Start 2023-01-04
Remain:2
start_count: 5
end_count: 13
Successfully for2023-01-04 00:00:00
Start 2021-05-15
Remain:1
start_count: 9
end_count: 10
Successfully for2021-05-15 00:00:00


In [2]:
df

,Tweet Date,Tweet Text
0,2022-09-23,Josealfredo found #bitcoin in a White vault at...
1,2022-09-23,Save your money while you still can.\n\n#Bitcoin
2,2022-09-23,WATCH: Get ready for the panic of your life be...
3,2022-09-23,http://METASECURITY.IN\n\nPremium #Domainname ...
4,2022-09-23,There's an awful lot of people around the worl...
...,...,...
85,2021-05-15,#Bitcoin is our best shot at clawing back some...
86,2021-05-15,So \n@elonmusk\n liked multiple tweets on this...
87,2021-05-15,#Bitcoin linear chart\nNext stop above 100K\n@...
88,2021-05-15,Can see the crypto ecosystem reaching that lev...


In [3]:
df.to_excel('Part_18.xlsx')

In [4]:
remaining_dates.to_series().to_csv('remaining_dates.csv', index=False)